In [34]:
from osgeo import gdal, ogr, osr
import os
import requests
import zipfile
import numpy as np 
import matplotlib.pyplot as plt
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
from pyresample import kd_tree,bilinear, geometry
from netCDF4 import Dataset
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, WrfProj)
import xarray as xr

In [6]:
def factors(nr):
    i = 2
    factors = []
    while i <= nr:
        if (nr % i) == 0:
            factors.append(i)
            nr = nr / i
        else:
            i = i + 1
    return factors

### Download Shapefile from NYC PLUTO Site

In [ ]:
# fname = 'nyc_mappluto_20v1_shp.zip'
# url = 'https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_mappluto_20v1_shp.zip'
# r = requests.get(url)
# open(fname , 'wb').write(r.content)

# path = os.getcwd()
# os.mkdir(path+"/Pluto")

# zip = zipfile.ZipFile(fname)
# zip.extractall("./Pluto")

### Load in Shapefile and rasterize

In [10]:
# Reads shapefile and save raster array to memory as np array 
vector_fn = './Pluto/MapPLUTO.shp'
# Define pixel_size and NoData value of new raster
# pixel_size = 30  #High res
# pixel_size = 101  #med res
pixel_size = 261  #Low res
NoData_value = 255
# Open the data source and read in the extent
source_ds = ogr.Open(vector_fn)
source_layer = source_ds.GetLayer()
source_srs = source_layer.GetSpatialRef()
layerDefinition = source_layer.GetLayerDefn()
# for i in range(layerDefinition.GetFieldCount()):
#     print(layerDefinition.GetFieldDefn(i).GetName())
x_min, x_max, y_min, y_max = source_layer.GetExtent()
# Create the destination data source
x_res = int((x_max - x_min) / pixel_size)
y_res = int((y_max - y_min) / pixel_size)
target_ds = gdal.GetDriverByName('MEM').Create('', x_res, y_res, gdal.GDT_Float32)
target_ds.SetGeoTransform((x_min, pixel_size, 0, y_max, 0, -pixel_size))
band = target_ds.GetRasterBand(1)
band.SetNoDataValue(NoData_value)
print(factors(x_res))
print(factors(y_res)) # need this for height histogram wy =5 & wx =5 when binning data, pick reasonable pixle window height and width 

[2, 5, 59]
[3, 3, 5, 13]


In [11]:
gdal.RasterizeLayer(target_ds, [1], source_layer, options = ["ATTRIBUTE=NumFloors"])
NumFloors = band.ReadAsArray()
gdal.RasterizeLayer(target_ds, [1], source_layer, options = ["ATTRIBUTE=GarageArea"])
GarageArea = band.ReadAsArray()
gdal.RasterizeLayer(target_ds, [1], source_layer, options = ["ATTRIBUTE=BldgArea"])
BldgArea = band.ReadAsArray()
gdal.RasterizeLayer(target_ds, [1], source_layer, options = ["ATTRIBUTE=LotArea"])
LotArea = band.ReadAsArray()
gdal.RasterizeLayer(target_ds, [1], source_layer, options = ["ATTRIBUTE=BldgFront"])
BldgFront = band.ReadAsArray()

NumFloors = NumFloors.astype(float)  
GarageArea =  GarageArea.astype(float)
BldgArea =  BldgArea.astype(float)
LotArea  = LotArea.astype(float)
BldgFront =  BldgFront.astype(float)

# Code below used to get lat lon grid for the rasters in the shapefile projection
srs = osr.SpatialReference()
srs.ImportFromWkt(source_srs.ExportToWkt())
srsLatLong = srs.CloneGeogCS()
ct = osr.CoordinateTransformation(srs, srsLatLong)
#                                         Lat = Y     Long = X
# Latitude and longitude coordinates are: 40.730610, -73.935242.
(lat_min, lon_min, height) = ct.TransformPoint(x_min, y_min)
(lat_max, lon_max, height) = ct.TransformPoint(x_max, y_max)
lon = np.linspace(lon_min, lon_max, num=NumFloors.shape[1], endpoint=True)
lat = np.linspace(lat_min, lat_max, num=NumFloors.shape[0], endpoint=True)
lon_grid, lat_grid = np.meshgrid(lon, lat)

### Compute UCP using raster data

In [14]:
""" Compute the UCP's needed to run BEPBEM. We use Gutierrez's
    methodology. We will compute the following UCP's:
    urban area fraction
    building height (and height histograms?)
    building surface to height ratio
    
    PLUTO Variables used: NumFloors, GarageArea, BldgArea, LotArea, BldgFront
"""


""" According to Burian et al. (2008) we define the plan area
    fraction as:
        planAreaFrac = planArea/lotArea
    where A_p is the total plan area of buildings. We compute A_p
    from PLUTO fields as:
        A_p = (buildArea - garageArea)/numFloors
"""
arr=BldgArea  - GarageArea 
arr[arr < 0] = 0

PlanArea = np.divide(arr, NumFloors , out=np.zeros_like(arr), where=NumFloors !=0)
planAreaFrac = np.divide(PlanArea, LotArea , out=np.zeros_like(PlanArea), where=LotArea !=0)
planAreaFrac[LotArea  < PlanArea] = 1

In [15]:
""" Compute building height from the number of floors in a
                building. Floor height can be changed (default 5 m)
"""
floorheight=5
buildheight = NumFloors *floorheight

In [16]:
""" Compute the building surface area to plan area ratio:
        surfplanratio = (2*buildheight/buildfront + 1)*buildfrac
    INPUT:
    buildhgt: building height in meters
    buildfront: building frontage in meters
    buildfrac: building plan area fraction
"""
buildwidth = BldgFront *.3048
b=np.divide(2*buildheight, buildwidth, out=np.zeros_like(2*buildheight), where=buildwidth!=0)
surfplanratio = (b + 1)*planAreaFrac

In [88]:
# Based off of this https://stackoverflow.com/questions/46013731/trying-to-calculate-the-mean-of-a-sliding-window-of-an-image-python
# Figuring out Building height histogram using a moving window based off the code from above
img = buildheight
# for 50 , 30 , 101 obtain from factors 
# wy =13 19 7
# wx =3 5 7 
# For pixle sz 261 this is the moving window width and height 
wy =5
wx =5

wy = wy or wx
y, x = img.shape
if x % wx != 0 or y % wy != 0:
    raise ValueError("Invalid window size.")
ny = y // wy
nx = x // wx
windowed = img.reshape((ny, wy, nx, wx))

def my_func(a): # special function to get histogram of height "Bin the data"
    bins=np.arange(0,80,5)
    hist_data=np.histogram(np.clip(a, bins[0], bins[-1]), bins=bins)
    hist_data_norm=np.true_divide(hist_data[0], hist_data[0].sum())
    return hist_data_norm

means_temp=np.rollaxis(windowed,3,2).reshape(ny,wx*wy,nx)
means_temp=np.apply_along_axis(my_func, 1, means_temp)
means_temp_SP=np.swapaxes(means_temp,1,2)
means_temp_RP = means_temp_SP.reshape(ny,1, nx,1, 15)
means_temp_tile = np.tile(means_temp_RP, (1, wy, 1, wx,1))
result = means_temp_tile.reshape((y, x,15))
result[result>.9]=0
# result[result == 0]=np.nan
buildheighthist=result
# np.savez('./pluto_array_data_FORMATED_coarse_res_px_261', planAreaFrac=planAreaFrac, buildheight=buildheight, surfplanratio=surfplanratio, buildheighthist=buildheighthist)

In [87]:
# # Load in data in you saved it previously
# npzfile = np.load('pluto_array_data_FORMATED_coarse_res_px_261.npz')
# planAreaFrac = npzfile['planAreaFrac']
# buildheight = npzfile['buildheight']
# surfplanratio = npzfile['surfplanratio']
# buildheighthist = npzfile['buildheighthist']
# lon = np.linspace(lon_min, lon_max, num=buildheighthist.shape[1], endpoint=True)
# lat = np.linspace(lat_min, lat_max, num=buildheighthist.shape[0], endpoint=True)
# lon_grid, lat_grid = np.meshgrid(lon, lat)

### Resample PLUTO Raster to WRF geo_em_d03 and export to nc file

In [ ]:
# Variables from pluto ncl script || Registry.EM_COMMON data || data from real init script showing how real extracts variables from URB_PARAM variable
#                 lu=a->LU_INDEX            # lup=b->LCLU_URB2D_NY
# planAreaFrac    lp=a->BUILD_AREA_FRACTION # lpp=b->LP_URB2D_NY   ||  LP_URB2D "BUILD_AREA_FRACTION" "BUILDING PLAN AREA DENSITY"                        ||  grid%LP_URB2D(i,j)   = grid%URB_PARAM(i,91,j)
# buildheight     bh=a->BUILD_HEIGHT        # bhp=b->HGT_URB2D_NY  ||  HGT_URB2D "BUILD_HEIGHT" "AVERAGE BUILDING HEIGHT WEIGHTED BY BUILDING PLAN AREA"  ||  grid%HGT_URB2D(i,j)  = grid%URB_PARAM(i,94,j)
# surfplanratio   lb=a->BUILD_SURF_RATIO    # lbp=b->LB_URB2D_NY   ||  LB_URB2D  "BUILD_SURF_RATIO" "BUILDING SURFACE AREA TO PLAN AREA RATIO"            ||  grid%LB_URB2D(i,j)   = grid%URB_PARAM(i,95,j)
# buildheighthist hi=a->HEIGHT_HISTOGRAMS   # hip=b->HI_URB2D_NY   ||  HI_URB2D  "HEIGHT_HISTOGRAMS" "DISTRIBUTION OF BUILDING HEIGHTS"                   ||  grid%HI_URB2D(i,k,j) = grid%URB_PARAM(i,k+117,j)

In [41]:
def resample_pluto_to_WRF(pluto_var,lon_grid,lat_grid,GEO_EM_variable): # special function to get histogram of height "Bin the data"
# Code below is used to resample planAreaFrac
    data_rs=np.flipud(pluto_var)
    orig_def = geometry.SwathDefinition(lons=lon_grid, lats=lat_grid)
    targ_def = geometry.SwathDefinition(lons=GEO_EM_variable.XLONG_M.data.squeeze(), lats=GEO_EM_variable.XLAT_M.data.squeeze())
    result_data_rs = kd_tree.resample_gauss(orig_def, data_rs ,targ_def, radius_of_influence=100, sigmas=100)
#     orig_def_obj=orig_def.compute_optimal_bb_area()
#     orig_def_bb=orig_def_obj.area_extent_ll
#     targ_def_obj=targ_def.compute_optimal_bb_area()
    return result_data_rs

In [86]:
# Need to provide wrf geo_em_d03 in order to proceed
GEO_EM_DATA = xr.open_dataset('geo_em.d03.nc')

GEO_EM_DATA.URB_PARAM.data[:,90,:,:]=resample_pluto_to_WRF(planAreaFrac,lon_grid,lat_grid,GEO_EM_DATA)
GEO_EM_DATA.URB_PARAM.data[:,93,:,:]=resample_pluto_to_WRF(buildheight,lon_grid,lat_grid,GEO_EM_DATA)
GEO_EM_DATA.URB_PARAM.data[:,94,:,:]=resample_pluto_to_WRF(surfplanratio,lon_grid,lat_grid,GEO_EM_DATA)

for idx, idy in zip(range(117,132), range(0,15)):
    GEO_EM_DATA.URB_PARAM.data[:,idx,:,:]=resample_pluto_to_WRF(buildheighthist[:,:,idy],lon_grid,lat_grid,GEO_EM_DATA)
    
GEO_EM_DATA.to_netcdf('geo_em.d03.nc.UPDATED')